# Topic models

We will use the `gensim` library to estimate topic models in Python. You will need to install the library:

In [ ]:
%pip install gensim

After installing, we can load the necessary functions:

In [1]:
import os
os.chdir('/Users/tcoan/git_repos/ncrm-spring-school')

import numpy as np
import pandas as pd
import gensim
from gensim import corpora
from gensim.models import LdaModel
from gensim.models.coherencemodel import CoherenceModel

# We will need to following packages to read/write data
import pickle
import csv

import warnings
warnings.filterwarnings("ignore")

Next, we need to find some text data to use and get it in a format that `gensim` likes. We will pull in some data on the Heartland Institute (i.e., a climate change skeptic think tank in the US):

In [2]:
with open('data/heartland.pkl', 'rb') as pfile:
    heartland = pickle.load(pfile)

Let's take a look at the first row to get a feel for the data:

In [ ]:
heartland[0]

How many documents does `heartland` include:

In [ ]:
len(heartland)

We will use the "text_postprocessed" field in our analysis. This field (or variable if you are a social scientist) takes the raw "text" and 1) converts to lowercase, 2) removes punctuation, and 3) removes stopwords. That's it! Note that a number of recent studies suggest that it is better to do as little pre-processing as possible when fitting topic models -- i.e., it is better to "post" process, rather than "pre" process. See Schofield et al. <a href = "http://www.cs.cornell.edu/~xanda/winlp2017.pdf">"Understanding Text Pre-Processing for Latent Dirichlet Allocation"</a>.

Next, we tokenize of text for `gensim` and create a gensim `Dictionary`:

In [ ]:
# Prepare data for gensim
documents = [row['text_postprocessed'] for row in heartland]
texts = [doc.split(' ') for doc in documents]

# Make gensim corpus object
dictionary = corpora.Dictionary(texts)

In [ ]:
dictionary

This dictionary just maps our tokens (words) from strings to integers. For instance, let's view the mapping for the word "alarmist":

In [ ]:
# Look up the integer ID if we have a token key
print(dictionary.token2id['alarmist'])

# And vice-a-versa
print(dictionary[10000])

In [ ]:
len(dictionary)

The next step is to "vectorize" our data based on the dictionary mapping created above. Here, we vectorize using the `doc2bow()` method (i.e., document to "bag of words"):

In [ ]:
# Gensim corpus object
corpus = [dictionary.doc2bow(text) for text in texts]

This `corpus` object holds the same information as the "document-term matrix" that we created in `sklearn`, but it is stored in a different (efficient) format:

In [ ]:
print(corpus[1000])

Next, we are ready to estimate our model:

In [ ]:
# Estimate!
k = 30
lda = LdaModel(corpus, num_topics=k, iterations=100, alpha = 'auto',
               id2word=dictionary, random_state=12345)

That's it! We can pull out the topic distribution ($\theta$) and the word distribution ($\phi$) as follows:

In [ ]:
# Documents on the rows and topic proportions on the columns
theta = lda.get_document_topics(corpus, minimum_probability = 0)

# Topics on the rows and words on the columns
phi = lda.get_topics()

In [ ]:
print(f'theta has {len(theta)} rows and {len(theta[0])}')
print(f'phi has {len(phi)} rows and {len(phi[0])}')

In [ ]:
theta[0]

Working with these matrices directly is a bit of a pain and ``gensim`` offers a number of methods to make examining your results easier. We'll look at some of those below, in the context of topic interpretation.

# Topic interpretation

We typically start our analysis of our topic model results by examining the topic **keywords**, examining the **assignment frequencies** of our topics, and analyzing the **semantic coherence** of our topics. We will extract the relevant information from our fitted `gensim` model to achieve each of these objectives.

## Keywords

We start the process of topic interpretation and validation by examining the topic **keywords**. These "keywords" are probable tokens under the model -- i.e., the tokens most often assigned to a particular topic. We can by using the `show_topic()` or `show_topics()` method for LDA objects:

In [ ]:
print(lda.show_topic(10, topn = 10))

In [ ]:
keys = lda.show_topics(num_topics=30, num_words = 5)
for key in keys:
    print(key)

I'm not crazy about gensim's default display of keywords. Instead, I like to use to extract and view the topics:

In [ ]:
# You will need to install prettytable to run this! E.g.,
# pip install prettytable
from prettytable import PrettyTable

def view_keywords(model, num_topics, num_words = 10, prettyprint = True):
    # Return keywords from gensim
    keywords = model.show_topics(num_topics = num_topics, 
                               num_words = num_words, 
                               formatted=False)

    # Reformat keyword results for easy viewing
    output = []
    for row in keywords:
        tokens = ' '.join([token[0] for token in row[1]])
        output.append([row[0], tokens])
    
    # Print a nicely formatted table
    if prettyprint:
        tbl = PrettyTable()
                
        # Column labels
        tbl.field_names = ["Topic ID", "Keywords"]
        
        # Populate table
        for row in output:
            tbl.add_row(row)
        
        # Output formatted table
        tbl.align = "l"
        print(tbl)
    
    return output

We can now use this function to view keywords in a format that doesn't hurt the eyes:

In [ ]:
keywords = view_keywords(lda, lda.num_topics, num_words = 10)

### How do we decide what a topic is about?

We read! That is, we often look at a handful of the most probable documents for each topic. Here's a function that does just that:

In [ ]:
def top_documents(content, topic_id, theta, n = 10):
    ''' Takes a topic id and the full document-topic 
        matrix (theta) and returns the indices for the 
        top n documents. '''
    
    x = theta[topic_id,:].todense()
    idx = np.argpartition(x[0,:], -n)[0,-n:]
    
    # Get sorted IDs
    idx_sorted = idx[0, np.argsort(-x[0,idx])].tolist()[0]
    
    # Find and return the top documents
    return [row for i,row in enumerate(content) if i in set(idx_sorted)]

In [ ]:
print('Extracting the full theta matrix. This can be slow!')
theta = gensim.matutils.corpus2csc(lda[corpus])

We can use our function to pull the top `n` most probable documents for a given topic:

In [ ]:
len(top_docs)

In [ ]:
top_docs = top_documents(heartland, 1, theta)
print(top_docs[1]['text'])

In [ ]:
top_docs[2]['text']

## Topic assignment frequency

Next, I like to extract on overall measure of the **importance** of a topic to a corpus. Here, we can define a function to extract the topic assignment frequencies:

In [ ]:
# Function to return the topic frequency
def topic_frequency(model, corpus, proportion = True, LOG_EVERY_N = 1000):
    ''' Takes a gensim model object and a corpus object
        and returns the number of words assigned to each
        topic. '''
    
    # Extract topic distributions
    theta = model.get_document_topics(corpus, minimum_probability = 0)
    
    # Extract number of words in each document
    n = [sum([row[1] for row in doc]) for doc in corpus]
    
    # Get topic assignments
    print('Extracting topic assignments for each token...')
    
    counts = []
    for i,row in enumerate(theta):
        # Extract topic assignemnt counts
        counts.append([round(el[1]*n[i]) for el in row])
        
        # Log progress
        if (i % LOG_EVERY_N) == 0:
            print('Finished processing %s documents' % i)
            
    # Convert to a numpy array
    counts_matrix = np.array(counts)
    
    # Sum down topics to get assignment totals
    assignments = np.sum(counts_matrix, axis = 0)
    
    if proportion:
        res = assignments/np.sum(assignments)
    else:
        res = assignments
    
    return res.tolist()

We can then get the assignment proportions by simply calling the function:

In [ ]:
# Extract topic assignments
assignments = topic_frequency(lda, corpus)

And we can print the assignment proportions out to get a sense of topic prevelance in our corpus:

In [ ]:
for i, row in enumerate(assignments):
    print('Topic %s = %s' % (i, round(row, 3)))

## Topic "quality"

How good our our topics? This is a tough question and there are many ways to answer it. Two common measures for examining topic quality are 1) **semantic coherence** and 2) topic **exclusivity**. Let's look at each in turn.

### Semantic coherence

One useful measure of topic quality is so-called semantic (or topic) coherence. It is easy to implement a wide-range of coherence measures in `gensim`:

In [ ]:
topics = [row[1].split(' ') for row in keywords]  

In [ ]:
# Pull out and tokenize the topic keywords
topics = [row[1].split(' ') for row in keywords]

# Estimate coherence. The 'u_mass' and 'c_v'
# methods are good to try.
co = CoherenceModel(topics=topics,
                    texts=texts,
                    dictionary=dictionary,
                    coherence='c_v')

# Extract semantic coherence for each topic
semantic = co.get_coherence_per_topic()

We can view the semantic coherence associated with each topic:

In [ ]:
for i, row in enumerate(semantic):
    print('Topic %s = %s' % (i, row))

And we can average over the topics to calculate the semantic coherence of our overall model:

In [ ]:
print('Model coherence = %s' % np.mean(semantic))

### Exclusivity

As you estimate topic models in your own research, you will notice that some topics include very similar words. For instance, for our Heartland Institute topic model above, there are quite of few "energy" topics. This is often a sign that we've estimated a model with too many topics (more on this below!).

One way to calculate **topic exclusivity** is to start with the "word" exclusivity for each topic keyword:

\begin{equation}
\frac{P(w_{i}|k=t)}{P(w_{i})}
\end{equation}

That is, we calculate the probability of each word ($w_{i}$) given a particular topic ($t$), divided by the overall (or marginal) probability of $w_{i}$. Here's a function that takes care of this:

In [ ]:
# Note you could get a DeprecationWarning here, depending on your
# version of IPython. Either ignore it or upgrade IPython:
# pip install --upgrade ipykernel

def exclusivity(model, dictionary, num_words = 10):
    # Return topic keywords
    topics = model.show_topics(model.num_topics, 
                               formatted=False,
                               num_words=num_words)
    
    # Get word distribution (phi)
    topic_word_matrix = model.get_topics()
    
    # Calculate the marginal probability of the word
    word_prob = np.sum(topic_word_matrix, axis=0)
    
    # Return exclusivity for each topic-word combination
    ex_matrix = topic_word_matrix / word_prob[None,:]
    
    # Parse topic keywords  
    keys_ = [[(k[0]) for k in topic[1]] 
             for topic in topics]
    
    # Return mean word exclusivity as a measure of "topic"
    # exclusvity
    return [np.mean([ex_matrix[topic[0], dictionary.token2id[k]] 
                     for k in keys_[topic[0]]])
            for topic in topics]


In [ ]:
ex = exclusivity(lda, dictionary)

# Print exclusivity for each topic
for i, row in enumerate(ex):
    print("Topic %s = %s" % (i, row))

In [ ]:
np.mean(ex)

## Writing the results to disk

While you could view coherence in the Python console, I typically prefer to write everything out to a CSV file:

In [ ]:
topics_to_write = []
for i,topic in enumerate(topics):
    topics_to_write.append({'topic_id': i, 
                             'topic_prop': assignments[i], 
                             'coherence': semantic[i], 
                             'exlusivity': ex[i], 
                             'keywords': ' '.join(topic)})
df_topics = pd.DataFrame(topics_to_write)

In [ ]:
# df_topics.sort_values('coherence', ascending=False)
df_topics.to_csv('/Users/tcoan/git_repos/notebooks/TopicModels/topics_heartland.csv')

# Topic visualization

One useful way to view topic models generated by `gensim` is using the `PyLDAvis` library. To install from jupyter, use the standard:

`!pip install pyldavis`

And then to run:

In [ ]:
import pyLDAvis.gensim as gensimvis
import pyLDAvis

vis_data = gensimvis.prepare(lda, corpus, dictionary)
pyLDAvis.display(vis_data)

# You save an HTML file with the same information, using
# pyLDAvis.save_html(vis_data,'vis.html')

# Selecting the topic number, *k*

As we discussed in class, there's not a great "automated" way to select the "optimal" topic number and I would argue that the whole notion of an "optimal" solution is misguided. There are, however, better and worse solutions. My preferred way to choose the topic number, *k*, is to combine a measure of **semantic coherence** with a qualitative inspection of a particular solution. I ask myself the following basic question: *what do we gain and what do we lose by increasing the overall number of topics*?

Here's an example of a loop that calculates and stores the relevant information:

In [ ]:
# Initialize parameters for the loop
start_k = 5
end_k = 30
step_k = 5
cv_method = 'u_mass'
top_n = 10
iterations = 50

results = []
for k in range(start_k, end_k, step_k):
    print("Fit k = %s" % k)
    model = LdaModel(corpus, num_topics=k, iterations=iterations,
               alpha = 'auto', id2word=dictionary, 
               random_state=12345)
    
    cm = CoherenceModel(model=model, 
                    texts=texts, 
                    dictionary=dictionary, 
                    coherence=cv_method,
                    topn=top_n)
    
    coherence = cm.get_coherence()
    print("Coherence = %s" % coherence)
    
    results.append({'model': model,
                    'coherence': [k, coherence]})


So based on the `u_mass` estimator, the "optimal" solution is somewhere near $k = 10$. Let's take a look at a view of these solutions:

In [ ]:
idx = 1
keywords = view_keywords(results[idx]['model'], results[idx]['model'].num_topics, num_words = 10)

# Extracting topic model data

Once you have a model that you are happy with, you can extact the <b>document-topic matrix</b> for future use. For instance, consider the following code:

In [ ]:
# Get the document-topic matrix "theta")
theta = lda.get_document_topics(corpus, minimum_probability = 0)

# Parse "theta" and store topic proportions
proportions = []
for row in theta:
    proportions.append([el[1] for el in row])

print(proportions[0])

In [ ]:
len(proportions[0])

This will provide a Python list of the relevant topic probabilities in the same order as the orginal `heartland` corpus. We can append these probabilities to our dataset and write to disk:

In [ ]:
# Define a little function to "flatten" a list
def flatten(list_of_lists):
    return [item for sublist in list_of_lists for item in sublist]

# Prepare file to write
data_to_write = []
for i,row in enumerate(proportions):
    meta = [heartland[i]['docid'], heartland[i]['date'], heartland[i]['title']]
    data_to_write.append(flatten([meta, row]))

# Write a CSV file
with open('heartland_data.csv', 'w') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerows(data_to_write)

## Plotting topic time series

Once we've extracted our data, what can we do with it? Often we are interested the dynamics of topic prevalence over time. As an example, say that we use the 15 topic solution and want to plot the average topic proportion on climate science over time. And to make things simple, assume that we want to plot the topic proportion at the yearly level. The first thing that we need to do is format our "date" variable and pull out the relevant year:

In [ ]:
# Import a module to deal with datetime
from datetime import datetime

# Let's strip the very first date to see how this works
datetime.strptime(data_to_write[0][1], '%m/%d/%Y').year

In [ ]:
data_to_write[0][1]

We just need to loop over our data and extact the year:

In [ ]:
for row in data_to_write:
    date = datetime.strptime(row[1], '%d/%m/%Y')
    row.append(date.year)

In [ ]:
print(data_to_write[0])

In [ ]:
import pandas as pd
df = pd.DataFrame(data_to_write)
df.head()

Let's give our data frame better variable names:

In [ ]:
# Make a list of column names
cnames = flatten([['docid', 'date', 'title'], ['topic' + str(i) 
                                               for i in range(30)], ['year']])

# Here's what the list looks like
print(cnames)

# Rename columns
df.columns = cnames
df.head()

Now we aggregate our topics by year using the average:

In [ ]:
# Split data groups by year
grouped = df.groupby('year')

# Aggregate groups based on the mean
df_ts = grouped.aggregate(np.mean)

# Let's take a look
df_ts.head()

Finally, we can plot the time series. We need to import plotting functionality from `matplotlib` and format in a pandas `Series`:

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt

# Format the pandas variable in a Series
ts = pd.Series(df_ts['topic1'])

And plot:

In [ ]:
ts.plot()

## Extensions to the LDA

We've discussed that one of the reasons that the LDA has been so influential is the ease with which it can be extended. Let's take a look a couple of extensions that could be very useful in your research. While there are a number of different implementations in Python available, we will be using the `tomotopy` library. 

In [ ]:
# !pip install tomotopy # install to use!
import tomotopy as tp

### Seeded LDA

The "seeded" or "guided" LDA offers a semi-supervised approach to "nudge" topics to a particular concept or theme by using a set of keywords. The way we do this in practice is by setting the priors on words in a way that pushes a word onto a topic. Let's start with the basic LDA in `tomotopy`:

In [ ]:
texts[0]

In [ ]:
# Instantiate the LDA model
model = tp.LDAModel(k=20)

# Add documents to the model
for text in texts:
    model.add_doc(text)

Train the model for 100 iterations:

In [ ]:
model.train(100)

In [ ]:
for k in range(model.k):
    print('Top 10 words of topic #{}'.format(k))
    print(model.get_topic_words(k, top_n=10))

In [ ]:
for k in range(model.k):
    print(f'Topic #{k}: {" ".join([keyword[0] for keyword in model.get_topic_words(k, top_n=10)])}')

Great, so that's the standard LDA -- how do we get the seeded version? We do so by setting the word priors:

In [ ]:
# We need to re-instantiate the LDA model
model = tp.LDAModel(k=20)

# Add documents to the model
for text in texts:
    model.add_doc(text)

model.set_word_prior('kyoto', [1.0 if k == 0 else 0.1 for k in range(20)])

In [ ]:
model.train(100)
for k in range(model.k):
    print(f'Topic #{k}: {" ".join([keyword[0] for keyword in model.get_topic_words(k, top_n=10)])}')

We can also easily add additional "seeds" to our model using something like:

In [ ]:
# We need to re-instantiate the LDA model
model = tp.LDAModel(k=20)

# Add documents to the model
for text in texts:
    model.add_doc(text)

# Set up seed words
keyword_seeds = [['kyoto', 'agreement', 'copenhagen'], 
                 ['energy', 'fossil', 'fuel'], 
                 ['gore', 'inconvenient']]

In [ ]:
for i,seeds in enumerate(keyword_seeds):
    for word in seeds:
        model.set_word_prior(word, [1.0 if k == i else 0.1 for k in range(20)])

In [ ]:
model.train(100)
for k in range(model.k):
    print(f'Topic #{k}: {" ".join([keyword[0] for keyword in model.get_topic_words(k, top_n=10)])}')

See the <a href="https://bab2min.github.io/tomotopy/v0.12.2/en/#tomotopy">tomotopy</a> hompage for more on the models they support, which include:

* Latent Dirichlet Allocation (LDAModel)
* Labeled LDA (LLDAModel)
* Partially Labeled LDA (PLDAModel)
* Supervised LDA (SLDAModel)
* Dirichlet Multinomial Regression (DMRModel)
* Generalized Dirichlet Multinomial Regression (GDMRModel)
* Hierarchical Dirichlet Process (HDPModel)
* Hierarchical LDA (HLDAModel)
* Multi Grain LDA (MGLDAModel)
* Pachinko Allocation (PAModel)
* Hierarchical PA (HPAModel)
* Correlated Topic Model (CTModel)
* Dynamic Topic Model (DTModel)
* Pseudo-document based Topic Model (PTModel).

And for examples of the applicaiton of several of these extentions in my publications, see:

* Seeded LDA: https://www.mdpi.com/2225-1154/7/3/45/htm
* Labeled LDA: https://www.cambridge.org/core/journals/politics-and-religion/article/political-speech-in-religious-sermons/53583EA3BD5F4223B5E31AA279698563


LDA-based models are no longer the only game in town and there are a number of exciting extensions that have their roots in language models. The `BERTopic` library ('https://github.com/MaartenGr/BERTopic) is the most well-developed version of this topic modelling strategy and can be used for:

* Guided topic modeling
* Supervised topic modeling
* Semi-supervised modeling
* Hierarchical modeling
* Dynamic topic modeling
* Etc., etc.!

I highly recommend checking the package out and comparing it to the classic LDA approach.